In [20]:
import pandas as pd
import pickle

paths = {
    "Sigmoid": "scores_Sigmoid.csv",
    "CMA-ES": "scores_CMA-ES.csv",
}
benchmarks = ["Sigmoid", "CMA-ES"]
best_selectors = ["MIS_Catch22_R_0.7", "DS_Catch22_R_0.8"]
template_scores = "aggregate_scores_{benchmark}.pickle"
template_scores_cis = "aggregate_score_cis_{benchmark}.pickle"

metrics = ["Median", "IQM", "Mean"]
template_perf = r"{lower:.3f}, \textbf{{{score:.3f}}}, {upper:.3f}"

def update_dict(scores: dict) -> dict:
    selector_key = "selector_" + best_selectors[i]
    scores = {k: v for k, v in scores.items() if k in ["full", "ISA", "random", selector_key]}
    if "oracle" in scores:
        scores["ISA"] = scores["oracle"]
        del scores["oracle"]
    scores["selector"] = scores[selector_key]
    del scores[selector_key]
    return scores

D = []
multi_index = []
for i, benchmark in enumerate(benchmarks):
    D_ = []
    fn = template_scores.format(benchmark=benchmark)
    with open(fn, "rb") as file:
        scores = pickle.load(file)
    fn = template_scores_cis.format(benchmark=benchmark)
    with open(fn, "rb") as file:
        scores_cis = pickle.load(file)

    scores = update_dict(scores)
    scores_cis = update_dict(scores_cis)

    keys = list(scores.keys())

    for k in keys:
        S = scores[k]
        CIs = scores_cis[k]
        
        data = {
            # "benchmark": benchmark,
            "origin": k,
        }
        # text = []
        for j, m in enumerate(metrics):
            # text.append(template_perf.format(lower=CIs[0,j], score=S[j], upper=CIs[1,j]))
            text = template_perf.format(lower=CIs[0,j], score=S[j], upper=CIs[1,j])
            data[(benchmark, m)] = text
            multi_index.append((benchmark, m))

        D_.append(pd.Series(data))
    D_ = pd.concat(D_, axis=1,).T
    D_.set_index(keys="origin", drop=True, inplace=True)
    D_.columns = pd.MultiIndex.from_tuples(D_.columns, names=["Benchmark", "Metric"])
    D.append(D_)

# D = pd.concat(D, axis=1,).T
# D.set_index(keys="origin", drop=True, inplace=True)
# D.columns = pd.MultiIndex.from_tuples(D.columns, names=["Benchmark", "Metric"])
D = D[0].join(D[1])

print(D)

latex_str = D.to_latex()

print()
print(latex_str)




Benchmark                       Sigmoid                                \
Metric                           Median                           IQM   
origin                                                                  
ISA        0.465, \textbf{0.471}, 0.474  0.470, \textbf{0.472}, 0.474   
full       0.438, \textbf{0.441}, 0.446  0.441, \textbf{0.443}, 0.446   
random     0.437, \textbf{0.443}, 0.445  0.439, \textbf{0.441}, 0.443   
selector   0.438, \textbf{0.443}, 0.446  0.440, \textbf{0.443}, 0.446   

Benchmark                                                      CMA-ES  \
Metric                             Mean                        Median   
origin                                                                  
ISA        0.475, \textbf{0.477}, 0.479  0.720, \textbf{0.730}, 0.781   
full       0.445, \textbf{0.447}, 0.448  0.713, \textbf{0.739}, 0.766   
random     0.443, \textbf{0.445}, 0.447  0.692, \textbf{0.715}, 0.734   
selector   0.443, \textbf{0.446}, 0.448  0.750, \t